In [1]:
!pip install fuzzywuzzy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import process
import warnings
import missingno as msno
warnings.simplefilter("ignore")

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
candy = pd.read_csv(r'/content/candy.csv')

In [4]:
candy.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


In [5]:
candy.drop(columns=['Internal ID','Unnamed: 113'], inplace = True)

In [6]:
candy.shape

(2460, 118)

In [7]:
candy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Columns: 118 entries, Q1: GOING OUT? to Click Coordinates (x, y)
dtypes: float64(4), object(114)
memory usage: 2.2+ MB


In [8]:
candy.describe()

,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
count,85.0,1362.0,99.0,67.0
mean,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0


**renaming the columns**



In [9]:
#getting the first and last index of Q6
indices = [i for i, col in enumerate(candy.columns) if isinstance(col, str) and 'Q6 | ' in col]
print(indices[0],indices[-1])

5 107


In [10]:
renaming=list(candy.columns[:5].str.split(': ').str[1])
renaming1 = list(candy.columns[5:108].str.split('Q6 \| ').str[1])
renaming2=list(candy.columns[108:117].str.split(': ').str[1])

In [11]:
#combine the lists
candy.columns = renaming + renaming1 + renaming2 + list(candy.columns[117:])

In [12]:
candy.columns = candy.columns.str.lower()

In [13]:
candy.columns

Index(['going out?', 'gender', 'age', 'country',
       'state, province, county, etc', '100 grand bar',
       'anonymous brown globs that come in black and orange wrappers\t(a.k.a. mary janes)',
       'any full-sized candy bar', 'black jacks', 'bonkers (the candy)',
       ...
       'joy other', 'despair other', 'other comments', 'dress', 'day',
       'media [daily dish]', 'media [science]', 'media [espn]',
       'media [yahoo]', 'click coordinates (x, y)'],
      dtype='object', length=118)

In [14]:
candy.duplicated().sum()

35

In [15]:
candy.isna().sum()

going out?                       110
gender                            41
age                               84
country                           64
state, province, county, etc     100
                                ... 
media [daily dish]              2375
media [science]                 1098
media [espn]                    2361
media [yahoo]                   2393
click coordinates (x, y)         855
Length: 118, dtype: int64

In [16]:
dup = candy.duplicated(keep = 'first')
candy[dup]


,going out?,gender,age,country,"state, province, county, etc",100 grand bar,anonymous brown globs that come in black and orange wrappers\t(a.k.a. mary janes),any full-sized candy bar,black jacks,bonkers (the candy),...,joy other,despair other,other comments,dress,day,media [daily dish],media [science],media [espn],media [yahoo],"click coordinates (x, y)"
36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
656,No,I'd rather not say,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,No,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
candy.dropna(axis = 'index', how = 'all',inplace = True)

In [18]:
candy.dtypes

going out?                       object
gender                           object
age                              object
country                          object
state, province, county, etc     object
                                 ...   
media [daily dish]              float64
media [science]                 float64
media [espn]                    float64
media [yahoo]                   float64
click coordinates (x, y)         object
Length: 118, dtype: object

In [19]:
candy.columns = candy.columns.astype('string')

In [20]:
candy['age'].unique()

array(['44', '49', '40', '23', nan, '53', '33', '43', '56', '64', '37',
       '59', '48', '54', '36', '45', '25', '34', '35', '38', '58', '50',
       '47', '16', '52', '63', '65', '41', '27', '31', '61', '46', '42',
       '62', '29', '39', '32', '28', '69', '67', '30', '22', '26', '51',
       '70', '24', '18', '19', 'Old enough', '57', '60', '66', '12',
       'Many', '55', '72', '?', '21', '11', 'no', '9', '68', '20', '6',
       '10', '71', '90', '13', '45-55', '312', '99', '7', 'hahahahaha',
       '88', '39.4', '74', '102', 'older than dirt', '17', '15', '8',
       '75', '5u', 'Enough', 'See question 2', '24-50', '14', 'Over 50',
       '100', '76', '1000', 'sixty-nine', '46 Halloweens.', 'ancient',
       '77', 'OLD', 'old', '73', '70 1/2', '1', 'MY NAME JEFF', '4',
       '59 on the day after Halloween', 'old enough', 'your mom',
       'I can remember when Java was a cool new language', '60+'],
      dtype=object)

In [21]:
def cleaning_age(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, str):
        value = value.strip().lower()
        if value in ['old enough', 'many', 'enough', 'see question 2', 'your mom', 'i can remember when java was a cool new language','no', '?', 'hahahahaha', 'my name jeff', 'your mom', 'ancient', 'old', 'older than dirt']:
            return np.nan
        elif '-' in value:
            return np.nan
        elif '/' in value:
            return np.nan
        elif '+' in value:
            return int(value.split('+')[0].strip())
        elif value.replace('.', '', 1).isdigit():
            return int(float(value))
        elif value.isdigit():
            return int(value)
        else:
            return np.nan

In [22]:
candy['age'] = candy['age'].apply(cleaning_age)

In [23]:
candy['age'].unique()

array([  44.,   49.,   40.,   23.,   nan,   53.,   33.,   43.,   56.,
         64.,   37.,   59.,   48.,   54.,   36.,   45.,   25.,   34.,
         35.,   38.,   58.,   50.,   47.,   16.,   52.,   63.,   65.,
         41.,   27.,   31.,   61.,   46.,   42.,   62.,   29.,   39.,
         32.,   28.,   69.,   67.,   30.,   22.,   26.,   51.,   70.,
         24.,   18.,   19.,   57.,   60.,   66.,   12.,   55.,   72.,
         21.,   11.,    9.,   68.,   20.,    6.,   10.,   71.,   90.,
         13.,  312.,   99.,    7.,   88.,   74.,  102.,   17.,   15.,
          8.,   75.,   14.,  100.,   76., 1000.,   77.,   73.,    1.,
          4.])

In [24]:
candy['age'] = candy['age'].astype('float')

In [25]:
candy['age'] = candy['age'].apply(lambda x: np.nan if x > 85 else x)

In [26]:
candy['age'].fillna(candy['age'].mean(),inplace = True)

In [27]:
candy['age'] = candy['age'].astype('int')

In [28]:
candy['going out?'].unique()

array(['No', nan, 'Yes'], dtype=object)

In [29]:
choices = ['Yes' , 'No']

In [30]:
candy['going out?'] = candy['going out?'].apply(lambda x: np.random.choice(choices) if pd.isna(x) else x)

In [31]:
candy['gender'].unique()

array(['Male', 'Female', "I'd rather not say", nan, 'Other'], dtype=object)

In [32]:
candy['gender'] = candy['gender'].fillna("I'd rather not say")

In [33]:
candy['country'].unique()

array(['USA ', 'USA', 'us', 'usa', nan, 'canada', 'Canada', 'Us', 'US',
       'Murica', 'United States', 'uk', 'United Kingdom', 'united states',
       'Usa', 'United States ', 'United staes',
       'United States of America', 'UAE', 'England', 'UK', 'canada ',
       'Mexico', 'United states', 'u.s.a.', 'USAUSAUSA', 'america', '35',
       'france', 'United States of America ', 'U.S.A.', 'finland',
       'unhinged states', 'Canada ', 'united states of america',
       'US of A', 'Unites States', 'The United States', 'North Carolina ',
       'Unied States', 'Netherlands', 'germany', 'Europe', 'Earth', 'U S',
       'u.s.', 'U.K. ', 'Costa Rica', 'The United States of America',
       'unite states', 'U.S.', '46', 'cascadia', 'Australia',
       'insanity lately', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", 'usas', 'Pittsburgh', '45', 'United State', '32',
       'France', 'australia', 'A', 'Can', 'Canae', 'New York',
       'Trumpistan', 'Ireland', 'United Sates', '

In [34]:
candy['country'] = candy['country'].str.strip().str.lower()

In [35]:
countries = pd.Series(['usa','america','united states','canada','uk','united kingdom','england','spain','mexico','france','netherlands','germany','europe','australia','costa rica','cascadia','australia','greece','ireland','korea','japan','south africa','iceland','scotland','denmark','indonesia','atlantis','china','taiwan','sweden'])

In [36]:
for country in countries:
  matches=process.extract(country,candy['country'],limit =candy.shape[0])
  for match in matches:
    if match[1] >= 70:
      candy.loc[candy['country'] == match[0] , 'country']=country

In [37]:
candy['country'].unique()

array(['usa', nan, 'cascadia', 'america', 'united states', 'uk',
       'united kingdom', 'uae', 'iceland', 'mexico', '35', 'france',
       'us of a', 'north carolina', 'netherlands', 'germany', 'europe',
       'earth', 'costa rica', '46', 'australia', 'insanity lately',
       'greece', 'usa? hard to tell anymore..', 'pittsburgh', '45', '32',
       'new york', 'trumpistan', 'korea', 'california', 'japan',
       'south africa', 'scotland', 'denmark', 'switzerland', 'ud',
       'new jersey', 'indonesia', 'atlantis', 'murrika', 'alaska',
       'soviet canuckistan', 'singapore', 'china', 'taiwan', 'hong kong',
       'spain', 'sweden', 'narnia', '1', 'subscribe to dm4uz3 on youtube',
       "i don't know anymore", 'fear and loathing'], dtype=object)

In [38]:
candy['country'] = candy['country'].replace(['america','united states','us of a','usa? hard to tell anymore..','new york','trumpistan','california','new jersey'],'usa')
candy['country'] = candy['country'].replace('united kingdom','uk')
x_country = ['35','earth','46','insanity lately','pittsburgh','45', '32','murrika','alaska','soviet canuckistan','narnia', '1', 'subscribe to dm4uz3 on youtube',"i don't know anymore", 'fear and loathing']
candy['country'] = candy['country'].replace(x_country,np.nan)
candy['country'] = candy['country'].apply(lambda x: np.random.choice(candy['country']) if pd.isna(x) else x)

In [39]:
candy['country'] = candy['country']

In [40]:
candy['country'] = candy['country'].astype('string')

In [41]:
candy['state, province, county, etc'].unique()

array(['NM', 'Virginia', 'or', 'exton pa', nan, 'Colorado ', 'ontario',
       'Ontario', 'Wa', 'Quebec', 'NY', 'California', 'Texas', 'NEW YORK',
       'CO', 'IN', 'MN', 'aberdeenshire', 'Alberta', 'Wisconsin, 53705',
       'RI', 'NC', 'Scotland', 'minnesota', 'Michigan', 'Florida',
       'New Jersey', 'VA', 'Colorado', 'Los Angeles, California', 'CA',
       'New York ', 'Chicago, Illinois', 'Nj', 'Wisconsin', 'Minnesota ',
       'London', 'mn', 'TEXAS', 'Illinois', 'District of Columbia',
       'Washington', 'Massachusetts ', 'OR', 'Montana', 'New York',
       'massachusetts', 'Abu Dhabi', 'TX', 'Arizona', 'Dc', 'yukon',
       'Pennsylvania ', 'MD', 'Ohio', 'Alberta ', 'OXFORDSHIRE', 'DC',
       'Iowa', 'Mn', 'TN', 'BC', 'Maryland', 'MI', 'Wisconsin ', 'alaska',
       'Ny', 'Colorado, Jefferson county ', 'Louisiana', 'pa',
       'CO, Larimer County', 'Nebraska ', 'Oregon', 'UT', 'ny',
       'Pennsylvania', 'CDMX', 'north carolina', 'Tennessee ', 'Texas ',
       'Knox cou

In [43]:
candy['dress'] = candy['dress'].apply(lambda x: np.random.choice(candy['dress']) if pd.isna(x) else x)

In [45]:
candy['day'] = candy['day'].fillna('other')

In [85]:
counts={}
for column in candy.columns[5:108]:
  counts[column] = candy[column].value_counts()
  print(counts)

Streaming output truncated to the last 5000 lines.
JOY        1439
MEH         276
DESPAIR      65
Name: count, dtype: int64, 'chardonnay': chardonnay
JOY        1040
MEH         360
DESPAIR     317
Name: count, dtype: int64, 'chick-o-sticks (we don’t know what that is)': chick-o-sticks (we don’t know what that is)
DESPAIR    644
MEH        571
JOY        299
Name: count, dtype: int64, 'chiclets': chiclets
DESPAIR    761
MEH        744
JOY        244
Name: count, dtype: int64, 'coffee crisp': coffee crisp
MEH        592
DESPAIR    541
JOY        474
Name: count, dtype: int64, 'creepy religious comics/chick tracts': creepy religious comics/chick tracts
DESPAIR    1406
JOY         246
MEH         104
Name: count, dtype: int64, 'dental paraphenalia': dental paraphenalia
DESPAIR    1433
MEH         244
JOY          91
Name: count, dtype: int64, 'dots': dots
DESPAIR    664
MEH        657
JOY        410
Name: count, dtype: int64, 'dove bars': dove bars
JOY        1176
MEH         474
DESPAIR

In [100]:
sorted_by_joy = sorted(counts.items(), key=lambda x: x[1]['JOY'], reverse=True)
top_10_joy = sorted_by_joy[:10]
i = 1
print("maximum 10 loved chocolates are:")
for key, sentiments in top_10_joy:
    print(i ,{key})
    i+=1

maximum 10 loved chocolates are:
1 {'any full-sized candy bar'}
2 {'reese’s peanut butter cups'}
3 {'kit kat'}
4 {'cash, or other forms of legal tender'}
5 {'twix'}
6 {'snickers'}
7 {'tolberone something or other'}
8 {'peanut m&m’s'}
9 {'lindt truffle'}
10 {'nestle crunch'}
